## Read .ghg metadata

In [12]:
import os
import time
import pandas as pd
from Parse_GHG import read_GHG
from ipywidgets import FloatProgress
from IPython.display import display

# Site = 'BB2'

# Values we expect to change
dynamicMetaData_Headers = ['altitude','canopy_height']

# Values we want to fix
staticMetaData_FloatHeaders = ['altitude', 'latitude', 'longitude']
staticMetaData_TextHeaders = ['site_name']

T1 = time.time()
print('Processing ghg files ')
f = FloatProgress(min=0, max=1) 
display(f) 
# Walk through the directory to find all "raw" folders

N_recs = 4#8*31
def find_new_GHG(Site,make_new=False):

    i = 0
    Dpath = 'C:\\highfreq\\'
    raw_dir = Dpath+Site+'\\raw\\'
    meta_dir = Dpath+Site+'\\metadata\\'
    meta_file = meta_dir+'GHGMetaData.csv'
    channel_file = meta_dir+'EP_Channels.csv'

    if os.path.isfile(meta_file) and make_new is False:
        Records = pd.read_csv(meta_file,parse_dates=['TimeStamp'])
        Channels = pd.read_csv(channel_file)#,parse_dates=['TimeStamp'])
        FileNames = Records['filename'].tolist()
    else:
        Records = pd.DataFrame()
        Channels = pd.DataFrame()
        FileNames = []
        if not os.path.exists(meta_dir):
            os.mkdir(meta_dir)
    

    for (root, dirs, files) in sorted(os.walk(raw_dir)):
        # if root.__contains__('raw') :
        for file in files:
            name, tag = file.split('.')
            # .ghg files are located at the end of each directory tree
            # Avoids reading any that might be misplaced elsewhere
            if tag == 'ghg' and len(dirs)==0 and file not in FileNames:  
                
                # if i <= 48*5:
                if i <= 2:
                    GHG_out = read_GHG(root,name)
                    Records = pd.concat(
                        [Records,GHG_out.Summary],
                        axis=0,
                        ignore_index=True
                    )
                    Channels = pd.concat(
                        [Channels,GHG_out.Channels],
                        axis=0,
                        ignore_index=True
                    )
                    i += 1
            f.value = i/len(files)
    return(Records,Channels,meta_dir,i)

Records,Channels,meta_dir,i = find_new_GHG('BB',make_new=False)

Records.to_csv(meta_dir+'GHGMetaData.csv',index=False)
Channels.to_csv(meta_dir+'EP_Channels.csv',index=False)

elapsed = time.time()-T1
print('Processing time ', elapsed) 
print('To inspect ', i, ' .ghg files')
print('Seconds/file ', elapsed/(i+1)) 

staticMetaData = Records[staticMetaData_FloatHeaders].astype(float).mean()
staticMetaData = pd.concat([Records[staticMetaData_TextHeaders].astype(str).mode().T[0],staticMetaData])
print(staticMetaData)

dynamicMetaData = Records.set_index('TimeStamp')[dynamicMetaData_Headers]
dynamicMetaData['date']=dynamicMetaData.index.date
dynamicMetaData['time']=dynamicMetaData.index.time
dynamicMetaData.to_csv(meta_dir+'dynamicMetaData.csv',index=False)
dynamicMetaData

# Processing time  65.87000775337219
# To inspect  241  .ghg files

Records['altitude'].astype(float).describe()


Processing ghg files 


FloatProgress(value=0.0, max=1.0)

Processing time  1.290328025817871
To inspect  3  .ghg files
Seconds/file  0.3225820064544678
site_name    BurnsBog2018
altitude             -3.8
latitude        49.129353
longitude     -122.984885
dtype: object


count    6.000000
mean    -3.800000
std      3.935988
min     -7.400000
25%     -7.225000
50%     -4.800000
75%     -0.125000
max      0.800000
Name: altitude, dtype: float64

In [20]:
# Channels = Channels.set_index('filename')
Channels.loc[(Channels.diff()).sum(axis=1)!=0].index


# Channels[(Channels.diff()!=0).sum(axis=0)]
# Channels.diff()f

Index([], dtype='object', name='filename')

In [50]:
all = []

for (root, dirs, files) in sorted(os.walk('C:\\highfreq\\BB\\raw\\')):
    # print(root,dirs,files)
    if files!=[]:
        all.append(files)

all_files = [file for sublist in all for file in sublist]

all_files.sort(reverse=True)

# print(all_files)

df = pd.DataFrame(data={'filename':all_files})

df['datetime']=df['filename'].str.split('_').str[0]
pd.to_datetime(df['datetime'])

,filename,datetime
0,2022-11-28T111150_LI-7200.ghg,2022-11-28T111150
1,2022-11-28T103000_LI-7200.ghg,2022-11-28T103000
2,2022-11-28T100000_LI-7200.ghg,2022-11-28T100000
3,2022-11-28T093000_LI-7200.ghg,2022-11-28T093000
4,2022-11-28T090000_LI-7200.ghg,2022-11-28T090000
...,...,...
7869,2022-06-17T140000_LI-7200.ghg,2022-06-17T140000
7870,2022-06-17T133000_LI-7200.ghg,2022-06-17T133000
7871,2022-06-17T130000_LI-7200.ghg,2022-06-17T130000
7872,2022-06-17T123000_LI-7200.ghg,2022-06-17T123000
